In [471]:
import serial
import io
import time
import threading
import sys
from pyModbusTCP.client import ModbusClient
from pyModbusTCP import utils
import re

#### Need to:
#####  1  Poll requests from Molly as they are requested and send responses
#####   a  Poll valve settings over TCP/IP and send back to molly (does it time out?  is the 'single serial link' of the 800 series?)
#####   b  Wait for request to change SL from Molly
#####   c Send new PV to TCP/IP

In [523]:
# data structures
# SMC stuff
modbus_registers = {'PV':100, 'SP': 102, 'SL':134, 'RR':136, 'OP':8}
smc_values = {'PV':0, 'SP': 0, 'SL':0, 'RR':0, 'OP':0}
# bisync stuff - back and forth to Molly
# SL is the command to send a new setpoint
bisync_commands = ['PV','SP','ER','SL','RR','OP']

In [455]:
# Classes to allow for reading and writing floats to the SMC
class FloatModbusClient(ModbusClient):
    def read_float(self, address, number=1):
        reg_l = self.read_holding_registers(address, number * 2)
        if reg_l:
            return [utils.decode_ieee(f) for f in utils.word_list_to_long(reg_l)][0]
        else:
            return None

    def write_float(self, address, floats_list):
        b32_l = [utils.encode_ieee(f) for f in floats_list]
        b16_l = utils.long_list_to_word(b32_l)
        return self.write_multiple_registers(address, b16_l)


In [454]:
def set_setpoint_eu808(ser, setpoint):
# This does not work - the SMC will not listen over the serial port
    if setpoint < 0 or setpoint > 300:
        print('Invalid range for setpoint, must be between 0 and 300')
        return
    if ser.is_open:
        msg = b'\x041100SP' + bytes(str(setpoint), encoding='ascii') + b'\x05'
        print(f"Sending '{msg}' to {ser.port}")
        #ser.write(bytes(msg, encoding='ascii'))
        ser.write(msg)
    else:
        print('Serial port {ser} is not open')
    time.sleep(0.02)
    return ser.readline()

def read_setpoint_from_eu808(ser, address):
# This one should not require a checksum
    # EOT(x04) UNIT_ADDRESS MNEMONIC ENQ (x05)
    if ser.is_open:
        add_str = str(address)
        add_formatted = bytes(2*add_str[0]+2*add_str[1], encoding='ascii')
        msg = b'\x04'+add_formatted+b'PV\x05'
        print(f"Sending {msg} to {ser.port}")
        #ser.write(bytes(msg, encoding='ascii'))
        ser.write(msg)
    else:
        print('Serial port {ser} is not open')
    time.sleep(0.2)
    return ser.readline()

def respond_to_molly(key, value):
    '''Send a single value to Molly'''       
    # STX (x02) MNEMONIC DATA ETX (x03) BCC
    key_formatted = bytes(key, encoding='ascii')
    if type(value)==str:
        round_value = value
    else:
        round_value = str(round(value,2))
    value_formatted = bytes(round_value, encoding='ascii')
    msg = b'\x02'+key_formatted+value_formatted+b'\x03'
    msg = msg + check_sum(msg)
    print(f'sending {str(msg)} to molly')
    if ser.is_open:
        ser.write(msg)
    else:
        print(f"Serial port {ser.port} not open")
            
def update_molly():
    '''Send all valve parameters to Molly'''
    for key in smc_values:
        respond_to_molly(key, smc_values[key])

In [578]:
# CJH functions to get and set values on the SMC via TCP/IP
def get_valve_values(verbose=True):
    ''' Ask for all values from the SMC'''
    c = FloatModbusClient(host='192.168.1.240', port=0, auto_open=True)
    # open or reconnect TCP to server
    if not c.is_open():
        if not c.open():
            print("unable to connect to "+SERVER_HOST+":"+str(SERVER_PORT))
    if c.is_open():
        for key in modbus_registers:
            smc_values[key] = c.read_float(modbus_registers[key], 1)
            #float_l = c.read_float(modbus_dict[key], 1)
            #print(f'Key: {key}  Value:{float_l}')
        c.close()
    if verbose:
        print(f'SMV values: {smc_values}') 
    
def set_valve(setpoint=0):
    '''Write valve setpoint to SMC'''
    if setpoint < 0 or setpoint > 300:
        return
    # open or reconnect TCP to server
    c = FloatModbusClient(host='192.168.1.240', port=0, auto_open=True)
    if not c.is_open():
        if not c.open():
            print("unable to connect to "+SERVER_HOST+":"+str(SERVER_PORT))
    if c.is_open():
        c.write_float(modbus_registers['SL'], [setpoint])
        c.close() 

In [579]:
def check_sum(message, debug=False):
    # Need this to finish the formatting of a message going back to Molly
    msg = message
    # Skip the first ETX (msg[0]), then XOR everything else
    result = msg[1]^msg[2]
    for char in msg[3:]:
        result = result ^ char
    if debug:
        print(f"message: {msg}, running check_sum: {result} (hex {hex(result)}) --> " + str(msg + bytes(chr(result),encoding='ascii')))
    return bytes(chr(result),encoding='ascii')

#for req in molly_change_requests:
#    check_sum(req[5:-1],debug=True)

message: b'\x02SL   0.0\x03', running check_sum: 18 (hex 0x12) --> b'\x02SL   0.0\x03\x12'
message: b'\x02SL 155.0\x03', running check_sum: 19 (hex 0x13) --> b'\x02SL 155.0\x03\x13'
message: b'\x02SL 300.0\x03', running check_sum: 17 (hex 0x11) --> b'\x02SL 300.0\x03\x11'


In [307]:
# note - this opens the port
#ser = serial.Serial('COM25', 9600, timeout=0.5)

In [539]:
# ser.open()
ser.close()

In [541]:
# test writing with a paperclip between pins 2 and 3 - seems to work fine
#COM24 is my RS232, 25 is my 484
ser = serial.Serial('COM24', 9600, timeout=0.25)

In [390]:
# Recognizing Molly requests
molly_change_requests = [b'\x040011\x02SL   0.0\x03\x12', b'\x040011\x02SL 155.0\x03\x13', b'\x040011\x02SL 300.0\x03\x11']
molly_read_requests = [b'\x040011SL\x05', b'\x040011SL\x05', b'\x040011PV\x05', 
                       b'\x040011SL\x05', b'\x040011SP\x05', b'\x040011SL\x05', b'\x040011OP\x05']
molly_requests = molly_change_requests + molly_read_requests
# straight from the EU808 handbook - answers are 0x39, 0x2e, 0x1c, 0x2c
# Note you always start with the 0x02, so it gets jammed in after the address
bcc_examples = [ b'\x02SW>0000\x03', b'\x02SP  44.\x03', b'\x02SL99\x03',b'\x02OP 61.9\x03']

In [ ]:
#Next steps: write an appropriate serial.threaded.Reader? protocol and see if it is handling molly requests as properly discretized events.  
#Must be an EOT type thing in there somewhere  

In [569]:
def parse_molly(requests, verbose=False, debug=False):
    r_enq = re.compile(b'\x04[0-9]+([A-Z]+)\x05')
    r_set = re.compile(b'\\x02SL([ 0-9A-Z.]+)\\x03')
    for msg in requests:
        
        if len(r_enq.findall(msg))>0:
            if verbose:
                print(f"Msg: {msg} ---> ENQ: {(r_enq.findall(msg))[0].decode('ascii')}  ---> SET: {r_set.findall(msg)}", end="\n")
            # do enquiry code here
            enquiry_string = (r_enq.findall(msg))[0].decode('ascii')
            if enquiry_string in bisync_commands:
                print(f'Sending {enquiry_string} value of {smc_values[enquiry_string]} to Molly  (prompted by {msg})')
                key = enquiry_string
                if not debug:
                    respond_to_molly(key, smc_values[key])            
            else:
                print(f'Invalid request: {enquiry_string}')
        
        if len(r_set.findall(msg))>0:
            if verbose:
                print(f"Msg: {msg} ---> ENQ: {r_enq.findall(msg)}  ---> SET: {(r_set.findall(msg))[0].decode('ascii').strip()}", end="\n")
            # do setting code here            
            setpoint = float((r_set.findall(msg))[0].decode('ascii'))
            print(f'Setting SMC setpoint to: {setpoint}  (prompted by {msg})')
            if not debug:
                set_valve(setpoint)
            
#parse_molly(molly_requests, verbose=False, debug=True)

In [577]:
def continuous_monitoring(runtime=None, verbose=False, debug=False):
    '''Continuously monitor Molly and translate requests to SMC'''
    start_time = time.time()
    elapsed_time = 0
    timed_out = False
    #flush the buffer - it's a pain to cut them all up
    clear_buffer = ser.readline()
    while timed_out is False:
        if runtime is not None:
            elapsed_time = time.time()-start_time
            if elapsed_time > runtime:
                timed_out = True
        data=ser.readline()
        if len(data) > 0:
            get_valve_values(verbose=False)
            print(data, len(data))
            parse_molly([data], verbose=False, debug=False)
            time.sleep(0.01)

{'PV': 33.0, 'SP': 33.0, 'SL': 33.0, 'RR': 25.0, 'OP': 11.0}
b'\x040011PV\x05' 8
Sending PV value of 33.0 to Molly  (prompted by b'\x040011PV\x05')
sending b'\x02PV33.0\x03\x1b' to molly
{'PV': 33.0, 'SP': 33.0, 'SL': 33.0, 'RR': 25.0, 'OP': 11.0}
b'\x040011SP\x05' 8
Sending SP value of 33.0 to Molly  (prompted by b'\x040011SP\x05')
sending b'\x02SP33.0\x03\x1e' to molly
{'PV': 33.0, 'SP': 33.0, 'SL': 33.0, 'RR': 25.0, 'OP': 11.0}
b'\x040011OP\x05' 8
Sending OP value of 11.0 to Molly  (prompted by b'\x040011OP\x05')
sending b'\x02OP11.0\x03\x02' to molly
{'PV': 33.0, 'SP': 33.0, 'SL': 33.0, 'RR': 25.0, 'OP': 11.0}
b'\x040011\x02SL  10.0\x03\x03' 16
Setting SMC setpoint to: 10.0  (prompted by b'\x040011\x02SL  10.0\x03\x03')
{'PV': 10.0, 'SP': 10.0, 'SL': 10.0, 'RR': 25.0, 'OP': 3.3333332538604736}
b'\x040011PV\x05' 8
Sending PV value of 10.0 to Molly  (prompted by b'\x040011PV\x05')
sending b'\x02PV10.0\x03\x1a' to molly
{'PV': 10.0, 'SP': 10.0, 'SL': 10.0, 'RR': 25.0, 'OP': 3.33333325

In [588]:
def reset_serial():
    global ser
    try:
        if ser.is_open:
            ser.close()
        else:
            pass
    except NameError:
        print(Error)
    ser = serial.Serial('COM24', 9600, timeout=0.25)
